In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from __future__ import division

import pickle
import os
import time

import scipy
import numpy as np
from stable_baselines3 import PPO

from matplotlib import pyplot as plt
import matplotlib as mpl

from mimi import envs
from mimi import utils
from mimi import user_models
from mimi import opt
from mimi import reward_models
from mimi import models

In [ ]:
sess = utils.make_tf_session(gpu_mode=False)

In [ ]:
user_id = 'sim'

In [ ]:
data_dir = os.path.join(utils.data_dir, 'cursor', user_id)
if not os.path.exists(data_dir):
  os.makedirs(data_dir)

In [ ]:
win_dims = np.array([1, 1]) * 1000
max_ep_len = 300
speed = 0.02
goal_dist_thresh = 0.05
reset_delay = 0
step_delay = 0.075
n_user_obs_dim = 2

In [ ]:
inv_dyn_model_init_args = [sess]
inv_dyn_model_init_kwargs = {
  'n_env_obs_dim': 2,
  'n_user_obs_dim': n_user_obs_dim,
  'n_act_dim': 2,
  'n_layers': 2,
  'layer_size': 64
}
inv_dyn_ens_init_kwargs = {
  'buffer_size': 100,
  'n_models': 4,
  'temp': 1e2
}
user_model = user_models.SimCursorUser(
  speed=speed, 
  inv_dyn_model_init_args=inv_dyn_model_init_args, 
  inv_dyn_model_init_kwargs=inv_dyn_model_init_kwargs,
  inv_dyn_ens_init_kwargs=inv_dyn_ens_init_kwargs,
  noise_std=1e-2,
  n_user_obs_dim=n_user_obs_dim
)

In [ ]:
env = envs.CursorEnv(
  sess,
  user_model,
  max_ep_len=max_ep_len,
  goal_dist_thresh=goal_dist_thresh,
  speed=speed,
  win_dims=win_dims,
  reset_delay=reset_delay
)

In [ ]:
mi_model_init_args = [sess]
mi_model_init_kwargs = {
  'n_env_obs_dim': env.n_env_obs_dim,
  'n_user_obs_dim': env.n_user_obs_dim,
  'n_act_dim': env.n_act_dim,
  'n_layers': 2,
  'layer_size': 64
}
mi_model_train_kwargs = {
  'iterations': 1000,
  'ftol': 1e-6,
  'learning_rate': 1e-3,
  'batch_size': 64,
  'val_update_freq': None,
  'verbose': False,
  'warm_start': False
}
reward_model = reward_models.MIRewardModel(
  env,
  mi_model_init_args,
  mi_model_init_kwargs,
  mi_model_train_kwargs
)

In [ ]:
def true_rew_of_rollout(rollout):
  goal = rollout[-1][-1]['goal']
  rews = [-np.linalg.norm(x[0][:2]-goal) for x in rollout]
  p = len(rollout) / env.max_ep_len
  return p * np.mean(rews) + (1-p) * rews[-1]
true_reward_model = lambda rollouts: np.mean([true_rew_of_rollout(rollout) for rollout in rollouts])

In [ ]:
intuitive_model = models.InvDynModel(*inv_dyn_model_init_args, **inv_dyn_model_init_kwargs)

In [ ]:
env_obses = np.random.random((10000, env.n_env_obs_dim))
user_obses = np.random.normal(0, 1, (10000, env.n_user_obs_dim))
user_obses /= np.linalg.norm(user_obses, axis=1, keepdims=True) + 1e-9
next_env_obses = env_obses + user_obses * env.speed
next_env_obses = np.maximum(0, next_env_obses)
next_env_obses = np.minimum(1, next_env_obses)
data = {
  'env_obses': env_obses,
  'user_obses': user_obses,
  'next_env_obses': next_env_obses
}

In [ ]:
data = utils.split_data(data, train_frac=0.9)

In [ ]:
inv_dyn_model_train_kwargs = {
  'iterations': 10000,
  'ftol': 1e-6,
  'learning_rate': 1e-3,
  'batch_size': 64,
  'val_update_freq': 1000,
  'verbose': True,
  'warm_start': False
}
intuitive_model.train(data, **inv_dyn_model_train_kwargs)

In [ ]:
user_model.inv_dyn_model.models[0] = intuitive_model

In [ ]:
def make_rand_policy():
  ang = np.random.random() * 2*np.pi
  return lambda obs: utils.rotate_vec(env.extract_user_obses(obs[np.newaxis])[0], ang)

n_policies = 5
n_rollouts_per_policy = 10

In [ ]:
baseline_rollouts = [[] for _ in range(n_policies)]

In [ ]:
for i in range(n_policies):
  policy = make_rand_policy()
  while len(baseline_rollouts[i]) < n_rollouts_per_policy:
    rollout = utils.run_ep(policy, env, render=True)
    baseline_rollouts[i].append(rollout)

In [ ]:
baseline_path = os.path.join(data_dir, 'baseline_rollouts.pkl')

In [ ]:
with open(baseline_path, 'wb') as f:
  pickle.dump(baseline_rollouts, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
true_reward_model(sum(baseline_rollouts, []))

In [ ]:
ppo_env = envs.RewardModelCursorEnv(
  sess,
  user_model,
  max_ep_len=max_ep_len,
  goal_dist_thresh=goal_dist_thresh,
  speed=speed,
  win_dims=win_dims,
  reset_delay=reset_delay,
  update_freq=10,
  buffer_size=10
)
ppo_env.set_reward_model(reward_model)

In [ ]:
ppo_kwargs = {
  'policy_kwargs': {'net_arch': [64, 64]}
}
ppo_model = PPO("MlpPolicy", ppo_env, verbose=1, **ppo_kwargs)

In [ ]:
ppo_model.learn(total_timesteps=200000)

In [ ]:
ppo_policy = lambda obs: ppo_model.predict(obs)[0]

In [ ]:
ppo_rollouts = []

In [ ]:
while len(ppo_rollouts) < 10:
  rollout = utils.run_ep(ppo_policy, ppo_env, render=True)
  ppo_rollouts.append(rollout)

In [ ]:
true_reward_model(ppo_rollouts)

In [ ]:
np.mean(ppo_env.reward_model.compute_step_rewards(ppo_rollouts))

In [ ]:
kernel_obses = user_obses[np.random.choice(np.arange(0, len(user_obses), 1), 100), :]

In [ ]:
gp_optimizer = opt.GP(
  env,
  reward_model,
  param_bounds=(-1., 1.),
  #param_bounds=(0, 2*np.pi),
  #n_policy_params=1,
  #W_from_w=(lambda w: np.array([[np.cos(w[0]), -np.sin(w[0])], [np.sin(w[0]), np.cos(w[0])]]))
  kernel_obses=kernel_obses
)

In [ ]:
gp_min_kwargs = {
  'n_initial_points': 5
}
ep_kwargs = {}
n_eps_per_pol = 10

In [ ]:
#x0, _, y0 = zip(*eval_data_of_pol)
#gp_min_kwargs.update({'x0': list(x0), 'y0': list(y0)})

In [ ]:
gp_policy, res = gp_optimizer.run(
  n_pols=500,
  n_eps_per_pol=n_eps_per_pol,
  gp_min_kwargs=gp_min_kwargs,
  ep_kwargs=ep_kwargs,
  reward_model_train_kwargs=mi_model_train_kwargs,
  verbose=True
)

In [ ]:
#eval_data_of_pol.extend(gp_optimizer.eval_data_of_pol)

In [ ]:
eval_data_of_pol = gp_optimizer.eval_data_of_pol

In [ ]:
gp_results_path = os.path.join(data_dir, 'gp_results.pkl')

In [ ]:
with open(gp_results_path, 'wb') as f:
  pickle.dump(eval_data_of_pol, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
data_dir = os.path.join(utils.data_dir, 'cursor')

In [ ]:
perf_evals = []
pols = []

for user_id in ['sim']:
  user_path = os.path.join(data_dir, str(user_id))
  if os.path.exists(user_path):
    baseline_path = os.path.join(user_path, 'baseline_rollouts.pkl')
    with open(baseline_path, 'rb') as f:
      baseline_rollouts = pickle.load(f)
    baseline_rollouts = sum(baseline_rollouts, [])
    baseline_true_rewards = true_reward_model(baseline_rollouts)
    
    gp_results_path = os.path.join(user_path, 'gp_results.pkl')
    with open(gp_results_path, 'rb') as f:
      eval_data_of_pol = pickle.load(f)
    true_rews_of_pol = [[true_reward_model([rollout]) for rollout in eval_data[1]] for eval_data in eval_data_of_pol]
    true_rew_of_pol = [true_reward_model(eval_data[1]) for eval_data in eval_data_of_pol]
    rew_of_pol = [eval_data[2] for eval_data in eval_data_of_pol]
    n_eps_of_pol = [len(eval_data[1]) for eval_data in eval_data_of_pol]
    
    true_rews = sum(true_rews_of_pol, [])
    perf_evals.append({
      'true_rews': true_rews, 
      'true_rew': true_rew_of_pol, 
      'rew': rew_of_pol,
      'n_eps': np.cumsum(n_eps_of_pol),
      'xs': np.cumsum(np.ones(len(true_rews)))-1,
      'baseline_true_reward': np.mean(baseline_true_rewards)
    })
    pols.append(eval_data_of_pol[-1][0])

In [ ]:
mpl.rcParams.update({'font.size': 12})

In [ ]:
pol_angs = [[pol[0] for pol in pols]]

N = 20
bottom = 8
max_height = 4

radii, theta = plt.hist(pol_angs, bins=20)[:2]
theta = np.mean(list(zip(theta[:-1], theta[1:])), axis=1)
width = (2*np.pi) / N

ax = plt.subplot(111, polar=True)
bars = ax.bar(theta, radii, width=width, color='orange')

plt.title('Emergent Interfaces')
plt.savefig(os.path.join(data_dir, 'sim-learned-int.pdf'), bbox_inches='tight')
plt.show()

In [ ]:
xs = sum([perf_eval['true_rew'] for perf_eval in perf_evals], [])
ys = sum([perf_eval['rew'] for perf_eval in perf_evals], [])
rho = scipy.stats.spearmanr(xs, ys)[0]
plt.title(r'True Reward vs. Mutual Information Reward ($\rho = %0.2f$)' % rho)
plt.xlabel('True Reward (Avg. Distance to Target)')
plt.ylabel(r"$\mathcal{I}(\mathbf{x}_t, (\mathbf{s}_t, \mathbf{s}_{t+1}))$")
plt.scatter(
  xs, 
  ys,
  color='orange',
  s=50,
  alpha=0.5
)
plt.xticks(fontsize=10)
plt.savefig(os.path.join(data_dir, 'sim-truerew-vs-mi.pdf'), bbox_inches='tight')
plt.show()

In [ ]:
plt.title('Mutual Information Reward')
plt.xlabel('Number of Online Training Episodes')
plt.ylabel(r"$\mathcal{I}(\mathbf{x}_t, (\mathbf{s}_t, \mathbf{s}_{t+1}))$")
utils.plot_perf_evals(perf_evals, 'n_eps', 'rew', label='MIMI (Ours)', smooth_win=1, color='teal')
plt.legend(loc='lower right')
plt.savefig(os.path.join(data_dir, 'sim-study-mi-vs-eps.pdf'), bbox_inches='tight')
plt.show()

In [ ]:
plt.title('User Study: 2D Cursor Control with Perturbed Mouse')
plt.xlabel('Number of Online Training Episodes')
plt.ylabel('True Reward (Avg. Distance to Target)')
utils.plot_perf_evals(perf_evals, 'xs', 'true_rews', label='MIMI (Ours)', smooth_win=1, color='orange')
plt.axhline(y=np.mean([perf['baseline_true_reward'] for perf in perf_evals]), linestyle='--', color='gray', label='Random Interfaces (Baseline)')
plt.axhline(y=-0.0726392950518139, linestyle='--', color='green', label='Oracle')
plt.legend(loc='lower right')
#plt.ylim([-0.14, None])
plt.savefig(os.path.join(data_dir, 'sim-truerew-vs-eps.pdf'), bbox_inches='tight')
plt.show()

In [ ]:
def plot_trajs(eval_data_of_pol):
  traj = np.array([x[0][:2] for eval_data in eval_data_of_pol for rollout in eval_data[1] for x in rollout])
  plt.scatter(traj[:, 0], traj[:, 1], alpha=0.25, linewidth=0, color='gray')
  plt.gca().set_aspect('equal', adjustable='box')
  plt.xticks([])
  plt.yticks([])
  
def plot_before_trajs(eval_data_of_pol):
  plt.title('<50 Training Episodes')
  plot_trajs(eval_data_of_pol[:5])
  
def plot_after_trajs(eval_data_of_pol):
  plt.title('>150 Training Episodes')
  plot_trajs(eval_data_of_pol[15:])

In [ ]:
with open(os.path.join(data_dir, 'sim', 'gp_results.pkl'), 'rb') as f:
  eval_data_of_pol = pickle.load(f)

In [ ]:
plot_before_trajs(eval_data_of_pol)
plt.savefig(os.path.join(data_dir, 'sim-before-trajs.pdf'), bbox_inches='tight')
plt.show()

In [ ]:
plot_after_trajs(eval_data_of_pol)
plt.savefig(os.path.join(data_dir, 'sim-after-trajs.pdf'), bbox_inches='tight')
plt.show()